# Imports

In [228]:
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import importlib
import contextlib
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from tqdm.notebook import tqdm
from IPython.display import display

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
import torch.nn.init as init
import copy
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Utils
import TEP_utils as TEP_utils
from PCA_chiang_utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

This notebook show cases how the metrics for the PCA on the data from Chiang works step-by-step. 

For a wrapped version see the function get_metrics_chiang_PCA_T2_and_Q. The function outputs the same results found by Chiang.

Also, the code from this function can be easily modified to work on any dataset, only requiring to change the function to load the dataset TEP_utils.load_dataset_Chiang_pca.

# Vizualization

Vizualization of a variable from falt 17.

In [208]:
# Dataset from Chiang
df_train_chiang, df_val_chiang, df_test_chiang = TEP_utils.load_dataset_Chiang_pca(fault=17)

In [209]:
# Dataset from Chiang
px.scatter(df_test_chiang['XMEAS(3)'])

# PCA ($T^2$)

Analysis of the PCA (T^2) explained variance.

In [329]:
# PCA
from sklearn.decomposition import PCA

pca, mean_and_std = get_pca(df_train_chiang, 1)
print(f'Num components: {pca.n_components_}')
print(f'Total Explained variance: {pca.explained_variance_ratio_.sum()}')

Num components: 52
Total Explained variance: 0.9999999999999999


In [331]:
# Cumsum explained variance
cumsum_explained_variance = np.cumsum(pca.explained_variance_ratio_)
px.line(x=range(1,1+len(cumsum_explained_variance)), y=cumsum_explained_variance, title='Cumsum explained variance')

In [213]:
# Get T^2 values
# Get PCA
pca, mean_and_std = get_pca(df_train_chiang, explained_variance=.54)

# Calculate T^2 values for validation set
T_2_values = calculate_T2(df_val_chiang, pca, mean_and_std)

# Get 95% and 99% percentiles
limit_95 = get_limit(T_2_values, 0.05)
limit_99 = get_limit(T_2_values, 0.01)

# Plot T^2 values
fig = px.scatter(y=T_2_values, title='T^2 values with 99% and 95% percentiles on validation set')
fig.add_shape(type='line', x0=0, x1=len(T_2_values), y0=limit_95, y1=limit_95, line=dict(color='red', width=2), name='95% percentile')
fig.add_shape(type='line', x0=0, x1=len(T_2_values), y0=limit_99, y1=limit_99, line=dict(color='blue', width=2), name='99% percentile')
fig.show()

# Calculate T^2 values for test set
T_2_values_test = calculate_T2(df_test_chiang, pca, mean_and_std)

# Plot T^2 values
fig = px.scatter(y=T_2_values_test, title='T^2 values with 99% and 95% percentiles on test set')
fig.add_shape(type='line', x0=0, x1=len(T_2_values_test), y0=limit_95, y1=limit_95, line=dict(color='red', width=2), name='95% percentile')
fig.add_shape(type='line', x0=0, x1=len(T_2_values_test), y0=limit_99, y1=limit_99, line=dict(color='blue', width=2), name='99% percentile')
fig.show()

In [197]:
# Load dataset
df_train_chiang, df_val_chiang, df_test_chiang = TEP_utils.load_dataset_Chiang_pca(fault=1)

# Calculate PCA from d00.dat
pca, mean_and_std = get_pca(df_train_chiang, explained_variance=0.54)

# Calculate T^2 values from d00_te.dat
T_2_values = calculate_T2(df_val_chiang, pca, mean_and_std)
# limit_99 = get_limit(T_2_values, 0.01)
limit_99 = np.percentile(T_2_values, 99, method='averaged_inverted_cdf')

for fault in range(1, 21+1):
    # Load dataset
    _, _, df_test_chiang = TEP_utils.load_dataset_Chiang_pca(fault=fault)

    # Detect faults
    T_2_values = calculate_T2(df_test_chiang, pca, mean_and_std)
    detected_as_faults = T_2_values >= limit_99

    # Faults gt
    ground_truth_faults = df_test_chiang['True IDV'].values==1

    # Calculate FAR for d(fault)_te.dat
    FAR = calculate_FAR(ground_truth_faults, detected_as_faults)
    MDR = calculate_MDR(ground_truth_faults, detected_as_faults)
    TTD = calculate_TTD(ground_truth_faults, detected_as_faults)
    print(f'Fault {fault}:\n\tFAR: {round(FAR, 3):.4f}, MDR: {MDR:.4f} TTD: {3*TTD}')
    


Fault 1:
	FAR: 0.0000, MDR: 0.0075 TTD: 21
Fault 2:
	FAR: 0.0000, MDR: 0.0200 TTD: 51
Fault 3:
	FAR: 1.0000, MDR: 1.0000 TTD: inf
Fault 4:
	FAR: 0.0300, MDR: 0.9600 TTD: inf
Fault 5:
	FAR: 0.0060, MDR: 0.7762 TTD: 48
Fault 6:
	FAR: 0.0000, MDR: 0.0112 TTD: 30
Fault 7:
	FAR: 0.0000, MDR: 0.0975 TTD: 3
Fault 8:
	FAR: 0.0000, MDR: 0.0338 TTD: 69
Fault 9:
	FAR: 0.6000, MDR: 0.9975 TTD: inf
Fault 10:
	FAR: 0.0000, MDR: 0.6663 TTD: 288
Fault 11:
	FAR: 0.0000, MDR: 0.7950 TTD: 912
Fault 12:
	FAR: 0.0000, MDR: 0.0288 TTD: 66
Fault 13:
	FAR: 0.0000, MDR: 0.0600 TTD: 147
Fault 14:
	FAR: 0.0000, MDR: 0.1688 TTD: 12
Fault 15:
	FAR: 0.0000, MDR: 0.9862 TTD: inf
Fault 16:
	FAR: 0.0230, MDR: 0.8375 TTD: 936
Fault 17:
	FAR: 0.0000, MDR: 0.2587 TTD: 87
Fault 18:
	FAR: 0.0000, MDR: 0.1138 TTD: 279
Fault 19:
	FAR: 0.0000, MDR: 0.9975 TTD: inf
Fault 20:
	FAR: 0.0000, MDR: 0.7063 TTD: 261
Fault 21:
	FAR: 0.0000, MDR: 0.7425 TTD: 1689


# PCA (Q)

Analysis of the PCA (Q) explained variance.

In [354]:
# Load dataset
df_train_chiang, df_val_chiang, df_test_chiang = TEP_utils.load_dataset_Chiang_pca(fault=16)

# Calculate PCA from d00.dat
pca, mean_and_std = get_pca(df_train_chiang, explained_variance=0.54)

# Get PCA
pca, mean_and_std = get_pca(df_train_chiang, explained_variance=.54)

# Get Q values
Q = calculate_Q(df_val_chiang, pca, mean_and_std)

# Get limit
limit_99 = get_limit(Q, 0.01)

# Plot Q values with 99% percentile
fig = px.scatter(y=Q, title='Q values with 99% and 95% percentiles on validation set')
limit_95 = get_limit(Q, 0.05)
fig.add_shape(type='line', x0=0, x1=len(Q), y0=limit_95, y1=limit_95, line=dict(color='red', width=2), name='95% percentile')
limit_99 = get_limit(Q, 0.01)
print('lim 99', limit_99)
fig.add_shape(type='line', x0=0, x1=len(Q), y0=limit_99, y1=limit_99, line=dict(color='blue', width=2), name='99% percentile')
fig.show()

# Calculate Q values for test set
Q_test = calculate_Q(df_test_chiang, pca, mean_and_std)
fig = px.scatter(y=Q_test)
fig.add_shape(type='line', x0=0, x1=len(Q_test), y0=limit_99, y1=limit_99, line=dict(color='red', width=2), name='99% percentile')
fig.show()

lim 99 50.858374070062325


In [378]:
# Load dataset
df_train_chiang, df_val_chiang, df_test_chiang = TEP_utils.load_dataset_Chiang_pca(fault=1)

# Calculate PCA from d00.dat
pca, mean_and_std = get_pca(df_train_chiang, explained_variance=0.54)

# Calculate T^2 values from d00_te.dat
Q_values_val = calculate_Q(df_val_chiang, pca, mean_and_std)
limit_99 = get_limit(Q_values_val, .01)

for fault in range(1, 21+1):
    # Load dataset
    _, _, df_test_chiang = TEP_utils.load_dataset_Chiang_pca(fault=fault)

    # Detect faults
    Q_values_test = calculate_Q(df_test_chiang, pca, mean_and_std)
    detected_as_faults = Q_values_test >= limit_99

    # Faults gt
    ground_truth_faults = df_test_chiang['True IDV'].values==1

    # Calculate FAR for d(fault)_te.dat
    FAR = calculate_FAR(ground_truth_faults, detected_as_faults)
    MDR = calculate_MDR(ground_truth_faults, detected_as_faults)
    TTD = calculate_TTD(ground_truth_faults, detected_as_faults)
    print(f'Fault {fault}:\n\tFAR: {round(FAR, 3):.4f}, MDR: {MDR:.4f} TTD: {TTD}')

    # if fault==16:
    #     break

Fault 1:
	FAR: 0.0010, MDR: 0.0025 TTD: 9
Fault 2:
	FAR: 0.0010, MDR: 0.0138 TTD: 36
Fault 3:
	FAR: 0.0000, MDR: 0.9938 TTD: inf
Fault 4:
	FAR: 0.0010, MDR: 0.0488 TTD: 9
Fault 5:
	FAR: 0.0050, MDR: 0.7462 TTD: 3
Fault 6:
	FAR: 0.0000, MDR: 0.0000 TTD: 3
Fault 7:
	FAR: 0.0000, MDR: 0.0000 TTD: 3
Fault 8:
	FAR: 0.0000, MDR: 0.0238 TTD: 60
Fault 9:
	FAR: 0.1330, MDR: 0.9838 TTD: inf
Fault 10:
	FAR: 0.0000, MDR: 0.6575 TTD: 147
Fault 11:
	FAR: 0.0020, MDR: 0.3638 TTD: 33
Fault 12:
	FAR: 0.0010, MDR: 0.0250 TTD: 24
Fault 13:
	FAR: 0.0000, MDR: 0.0450 TTD: 111
Fault 14:
	FAR: 0.0000, MDR: 0.0000 TTD: 3
Fault 15:
	FAR: 0.0000, MDR: 0.9725 TTD: inf
Fault 16:
	FAR: 0.0000, MDR: 0.7575 TTD: 591
Fault 17:
	FAR: 0.0010, MDR: 0.1113 TTD: 75
Fault 18:
	FAR: 0.0010, MDR: 0.1013 TTD: 252
Fault 19:
	FAR: 0.0000, MDR: 0.8862 TTD: inf
Fault 20:
	FAR: 0.0000, MDR: 0.5537 TTD: 261
Fault 21:
	FAR: 0.0060, MDR: 0.5700 TTD: 855


In [348]:
fig = px.scatter(ground_truth_faults)
fig.add_scatter(y=detected_as_faults, mode='markers')

# Final Results

Runs the both PCAs for all the faults.

In [289]:
def round_point_5_up(x, decimal):
    return np.floor(x*10**decimal + 0.5) / 10**decimal

In [379]:
results_MDR, results_FAR, results_TTD = get_metrics_chiang_PCA_T2_and_Q(explained_variance=.54)

print('MDR:')
display(round_point_5_up(results_MDR.astype(float), 3))

print('TTD:')
display(round_point_5_up(results_TTD.astype(float), 3))

print('FAR:')
display(round_point_5_up(results_FAR.astype(float), 3))

MDR:


,PCA (T²),PCA (Q)
1,0.008,0.003
2,0.020,0.014
3,0.994,0.990
4,0.943,0.034
5,0.771,0.743
6,0.011,0.000
7,0.064,0.000
8,0.034,0.024
9,0.986,0.981
10,0.640,0.634


TTD:


,PCA (T²),PCA (Q)
1,21.0,9.0
2,51.0,36.0
3,inf,inf
4,inf,9.0
5,48.0,3.0
6,30.0,3.0
7,3.0,3.0
8,69.0,60.0
9,inf,inf
10,288.0,147.0


FAR:


,PCA (T²),PCA (Q)
1,0.000,0.001
2,0.001,0.001
3,0.167,0.200
4,0.021,0.001
5,0.005,0.005
6,0.000,0.000
7,0.000,0.000
8,0.000,0.003
9,0.267,0.118
10,0.003,0.000
